ReentrantLock

A reentrant mutual exclusion Lock with the same basic behavior and semantics as the implicit monitor lock accessed using synchronized methods and statements, but with extended capabilities.
A ReentrantLock is owned by the thread last successfully locking, but not yet unlocking it. A thread invoking lock will return, successfully acquiring the lock, when the lock is not owned by another thread. The method will return immediately if the current thread already owns the lock. This can be checked using methods isHeldByCurrentThread(), and getHoldCount().

The constructor for this class accepts an optional fairness parameter. When set true, under contention, locks favor granting access to the longest-waiting thread. Otherwise this lock does not guarantee any particular access order. Programs using fair locks accessed by many threads may display lower overall throughput (i.e., are slower; often much slower) than those using the default setting, but have smaller variances in times to obtain locks and guarantee lack of starvation. Note however, that fairness of locks does not guarantee fairness of thread scheduling. Thus, one of many threads using a fair lock may obtain it multiple times in succession while other active threads are not progressing and not currently holding the lock. Also note that the untimed tryLock method does not honor the fairness setting. It will succeed if the lock is available even if other threads are waiting.

It is recommended practice to always immediately follow a call to lock with a try block, most typically in a before/after construction such as:

 class X {
   private final ReentrantLock lock = new ReentrantLock();
   // ...

   public void m() {
     lock.lock();  // block until condition holds
     try {
       // ... method body
     } finally {
       lock.unlock()
     }
   }
 }
 
In addition to implementing the Lock interface, this class defines methods isLocked and getLockQueueLength, as well as some associated protected access methods that may be useful for instrumentation and monitoring.

Serialization of this class behaves in the same way as built-in locks: a deserialized lock is in the unlocked state, regardless of its state when serialized.

This lock supports a maximum of 2147483647 recursive locks by the same thread. Attempts to exceed this limit result in Error throws from locking methods.

# Lock接口：
## public interface Lock
     Lock接口，相比于使用synchronized关键字，提供了更多互斥锁的操作。他允许更加灵活的结构，并且可能与隐式锁的使用有很大区别。
一个锁对象 为多个线程访问共享资源提供控制，通常，锁提供了互斥的资源访问方式：每次只有一个线程可以持有锁，每个对于共享资源的访问，都需要先持有该锁。然而。某些所允许并发的访问共享资源，如读写锁的读锁。<br/>
    同步方法或语句的使用，提供了对与每个对象关联的隐式监视器锁的访问，但是强制所有锁获取和释放，都以块结构的方式进行:当一个代码获取多个锁之后，必须以其获取锁相反的顺序，逐个释放锁，并且所有的锁都要在他们获取锁的同一代码范围内释放。

The use of synchronized methods or statements provides access to the implicit monitor lock associated with every object, but forces all lock acquisition and release to occur in a block-structured way: when multiple locks are acquired they must be released in the opposite order, and all locks must be released in the same lexical scope in which they were acquired.

虽然同步方法的范围机制，为程序使用管程锁提供了遍历，也有利于避免锁相关的编程错误，但是这种方式在某些场合缺乏灵活性。譬如，有些算法要求遍历并发访问数据结构，要求使用移交锁，或者链锁。Lock接口的实现类可以提供类似功能。
While the scoping mechanism for synchronized methods and statements makes it much easier to program with monitor locks, and helps avoid many common programming errors involving locks, there are occasions where you need to work with locks in a more flexible way. For example, some algorithms for traversing concurrently accessed data structures require the use of "hand-over-hand" or "chain locking": you acquire the lock of node A, then node B, then release A and acquire C, then release B and acquire D and so on. Implementations of the Lock interface enable the use of such techniques by allowing a lock to be acquired and released in different scopes, and allowing multiple locks to be acquired and released in any order.

灵活性的提高是有代价的，失去了代码范围的控制，也同时造成了释放锁的操作需要客户端来完成。一般都以以下方式使用锁：
With this increased flexibility comes additional responsibility. The absence of block-structured locking removes the automatic release of locks that occurs with synchronized methods and statements. In most cases, the following idiom should be used:

     Lock l = ...;
     l.lock();
     try {
         // access the resource protected by this lock
     } finally {
         l.unlock();
     }
 当获取锁和释放锁存在于不同的代码范围中时，必须格外注意，加锁与释放锁的代码都必须被执行。所有一般使用 try--finally代码块来实现。
When locking and unlocking occur in different scopes, care must be taken to ensure that all code that is executed while the lock is held is protected by try-finally or try-catch to ensure that the lock is released when necessary.
锁的实现提供了同步方法以外的其他功能:
* 非阻塞获取锁tryLock(
* 获取可中断锁lockInterruptibly()
* 设置获取锁的超时时间tryLock(long, TimeUnit)
Lock implementations provide additional functionality over the use of synchronized methods and statements by providing a non-blocking attempt to acquire a lock (tryLock()), an attempt to acquire the lock that can be interrupted (lockInterruptibly(), and an attempt to acquire the lock that can timeout (tryLock(long, TimeUnit)).

锁的实现也可以提供与隐式同步方法不同的行为和语义，如：保证排序，不可重入，死锁检测。如果实现提供了这种专门化的语义，那么实现必须记录这些语义。
A Lock class can also provide behavior and semantics that is quite different from that of the implicit monitor lock, such as guaranteed ordering, non-reentrant usage, or deadlock detection. If an implementation provides such specialized semantics then the implementation must document those semantics.
锁实例也只是普通对象，也可以在锁实例上同步，但是为了避免歧义，不建议这么做。
Note that Lock instances are just normal objects and can themselves be used as the target in a synchronized statement. Acquiring the monitor lock of a Lock instance has no specified relationship with invoking any of the lock() methods of that instance. It is recommended that to avoid confusion you never use Lock instances in this way, except within their own implementation.

Except where noted, passing a null value for any parameter will result in a NullPointerException being thrown.

## Memory Synchronization
所有锁的实现都器强制进行内存同步，这是由内置管程锁在JLS中规定的。
All Lock implementations must enforce the same memory synchronization semantics as provided by the built-in monitor lock, as described in section 17.4 of The Java™ Language Specification:
一个成功的加锁操作，与一个成功的加锁动作有同样的内存同步效果。
一个成功的释放锁操作，与一个成功的释放锁动作有同样的内存同步效果。
A successful lock operation has the same memory synchronization effects as a successful Lock action.
A successful unlock operation has the same memory synchronization effects as a successful Unlock action.
不成功的加锁和释放锁操作，以及重入的加锁，释放锁操作，不要求同样的内存同步效果。
Unsuccessful locking and unlocking operations, and reentrant locking/unlocking operations, do not require any memory synchronization effects.
## Implementation Considerations
关于实现的思考
锁获取的三种方式（中可断，不可中断，超时设置）可能在性能特征，排序保证，以及其他实现方面有所不同。甚至，中断正在获取锁的操作（或其他功能），可能在某些实现内是不支持的。一个实现并不要求实现所有三种获取锁的方式，也不需要必须实现可中断的锁获取方法。一个实现只需要清晰地记录每个锁定方法提供的语义和保证。他必须遵守该接口中定义的中断语义规则，锁的可中断获取，要么全部支持，或者只提供方法入口。
The three forms of lock acquisition (interruptible, non-interruptible, and timed) may differ in their performance characteristics, ordering guarantees, or other implementation qualities. Further, the ability to interrupt the ongoing acquisition of a lock may not be available in a given Lock class. Consequently, an implementation is not required to define exactly the same guarantees or semantics for all three forms of lock acquisition, nor is it required to support interruption of an ongoing lock acquisition. An implementation is required to clearly document the semantics and guarantees provided by each of the locking methods. It must also obey the interruption semantics as defined in this interface, to the extent that interruption of lock acquisition is supported: which is either totally, or only on method entry.
中断一般意味着取消，并且检查中断常常是不常见的，因此一个锁实现，相对于普通方法，可以偏向一个可中断调用。这适用于即使是在另一个动作发生中断可能会阻塞该线程的情况下，一个实现应该记录这种实现。
As interruption generally implies cancellation, and checks for interruption are often infrequent, an implementation can favor responding to an interrupt over normal method return. This is true even if it can be shown that the interrupt occurred after another action may have unblocked the thread. An implementation should document this behavior.

##  methods

### 	lock() 
获取锁：如果当前锁被占用，则获取锁的线程被阻塞，不能被调度，会在获取到锁之前，一直处于休眠状态。
Acquires the lock.
If the lock is not available then the current thread becomes disabled for thread scheduling purposes and lies dormant until the lock has been acquired.
实现：
一个锁的实现可能可以检测出锁使用的错误（可能造成死锁的），这时，会抛出一个未受检查的异常。锁的具体实现必须记录在doc中。
Implementation Considerations

A Lock implementation may be able to detect erroneous use of the lock, such as an invocation that would cause deadlock, and may throw an (unchecked) exception in such circumstances. The circumstances and the exception type must be documented by that Lock implementation.
### 	lockInterruptibly()
尝试获取锁，除非当前线程被中断。
如果锁可用，则立刻返回。
如果锁被占用，那么当前线程被阻塞，除非：
* 锁被当前线程获取
* 其他线程中断了当前线程，且支持中断获取锁操作

cquires the lock unless the current thread is interrupted.
Acquires the lock if it is available and returns immediately.

If the lock is not available then the current thread becomes disabled for thread scheduling purposes and lies dormant until one of two things happens:

The lock is acquired by the current thread; or
Some other thread interrupts the current thread, and interruption of lock acquisition is supported.
如果当前线程：
* 将自己的中断状态设置为该方法的入口，或者
* 在获取锁时被中断
那么就支持获取锁时被中断的功能，当被中断时，抛出InterruptedException，并且该线程的中断状态被清空。
If the current thread:

has its interrupted status set on entry to this method; or
is interrupted while acquiring the lock, and interruption of lock acquisition is supported,
then InterruptedException is thrown and the current thread's interrupted status is cleared.

实现：
中断获取锁的方法，在某些实现中是不可能的。并且支持该功能的实现中，该操作也可能是代价高昂的。编程人员应该了解这种情况
肯能会偏向应答中断，相比于普通方法返回
可能检测使用错误
Implementation Considerations

The ability to interrupt a lock acquisition in some implementations may not be possible, and if possible may be an expensive operation. The programmer should be aware that this may be the case. An implementation should document when this is the case.

An implementation can favor responding to an interrupt over normal method return.

A Lock implementation may be able to detect erroneous use of the lock, such as an invocation that would cause deadlock, and may throw an (unchecked) exception in such circumstances. The circumstances and the exception type must be documented by that Lock implementation.

Throws:
InterruptedException - if the current thread is interrupted while acquiring the lock (and interruption of lock acquisition is supported).

###  newCondition
### tryLock()
只有当锁可用时，才获取锁，获取锁时，立刻返回true，若此刻锁不可用，则返回false，该方法非阻塞。
Acquires the lock only if it is free at the time of invocation.
Acquires the lock if it is available and returns immediately with the value true. If the lock is not available then this method will return immediately with the value false.

A typical usage idiom for this method would be:

      Lock lock = ...;
      if (lock.tryLock()) {
          try {
              // manipulate protected state
          } finally {
              lock.unlock();
          }
      } else {
          // perform alternative actions
      }
 
This usage ensures that the lock is unlocked if it was acquired, and doesn't try to unlock if the lock was not acquired.
### tryLock(long time, TimeUnit unit)
尝试获取锁，如果锁在指定时间范围内可用，且当前线程未被中断。
若该锁可用，则立刻返回true，如果锁被占用，那么当前线程被阻塞，除非：
* 锁被当前线程获取
* 获取锁的线程被中断，且支持获取锁中断功能
* 等待时间用尽
若获取到锁，则返回true。
Acquires the lock if it is free within the given waiting time and the current thread has not been interrupted.
If the lock is available this method returns immediately with the value true. If the lock is not available then the current thread becomes disabled for thread scheduling purposes and lies dormant until one of three things happens:

The lock is acquired by the current thread; or
Some other thread interrupts the current thread, and interruption of lock acquisition is supported; or
The specified waiting time elapses
If the lock is acquired then the value true is returned.
如果当前线程：

If the current thread:

has its interrupted status set on entry to this method; or
is interrupted while acquiring the lock, and interruption of lock acquisition is supported,
then InterruptedException is thrown and the current thread's interrupted status is cleared.
If the specified waiting time elapses then the value false is returned. If the time is less than or equal to zero, the method will not wait at all.

Implementation Considerations

The ability to interrupt a lock acquisition in some implementations may not be possible, and if possible may be an expensive operation. The programmer should be aware that this may be the case. An implementation should document when this is the case.

An implementation can favor responding to an interrupt over normal method return, or reporting a timeout.

A Lock implementation may be able to detect erroneous use of the lock, such as an invocation that would cause deadlock, and may throw an (unchecked) exception in such circumstances. The circumstances and the exception type must be documented by that Lock implementation.

Parameters:
time - the maximum time to wait for the lock
unit - the time unit of the time argument
Returns:
true if the lock was acquired and false if the waiting time elapsed before the lock was acquired
Throws:
InterruptedException - if the current thread is interrupted while acquiring the lock (and interruption of lock acquisition is supported)

 # Condition
 
 ## interface
 
 condition因子
 Condition factors out the Object monitor methods (wait, notify and notifyAll) into distinct objects to give the effect of having multiple wait-sets per object, by combining them with the use of arbitrary Lock implementations. Where a Lock replaces the use of synchronized methods and statements, a Condition replaces the use of the Object monitor methods.
Conditions (also known as condition queues or condition variables) provide a means for one thread to suspend execution (to "wait") until notified by another thread that some state condition may now be true. Because access to this shared state information occurs in different threads, it must be protected, so a lock of some form is associated with the condition. The key property that waiting for a condition provides is that it atomically releases the associated lock and suspends the current thread, just like Object.wait.

A Condition instance is intrinsically bound to a lock. To obtain a Condition instance for a particular Lock instance use its newCondition() method.

As an example, suppose we have a bounded buffer which supports put and take methods. If a take is attempted on an empty buffer, then the thread will block until an item becomes available; if a put is attempted on a full buffer, then the thread will block until a space becomes available. We would like to keep waiting put threads and take threads in separate wait-sets so that we can use the optimization of only notifying a single thread at a time when items or spaces become available in the buffer. This can be achieved using two Condition instances.

 class BoundedBuffer {
   final Lock lock = new ReentrantLock();
   final Condition notFull  = lock.newCondition(); 
   final Condition notEmpty = lock.newCondition(); 

   final Object[] items = new Object[100];
   int putptr, takeptr, count;

   public void put(Object x) throws InterruptedException {
     lock.lock();
     try {
       while (count == items.length)
         notFull.await();
       items[putptr] = x;
       if (++putptr == items.length) putptr = 0;
       ++count;
       notEmpty.signal();
     } finally {
       lock.unlock();
     }
   }

   public Object take() throws InterruptedException {
     lock.lock();
     try {
       while (count == 0)
         notEmpty.await();
       Object x = items[takeptr];
       if (++takeptr == items.length) takeptr = 0;
       --count;
       notFull.signal();
       return x;
     } finally {
       lock.unlock();
     }
   }
 }
 
(The ArrayBlockingQueue class provides this functionality, so there is no reason to implement this sample usage class.)
A Condition implementation can provide behavior and semantics that is different from that of the Object monitor methods, such as guaranteed ordering for notifications, or not requiring a lock to be held when performing notifications. If an implementation provides such specialized semantics then the implementation must document those semantics.

Note that Condition instances are just normal objects and can themselves be used as the target in a synchronized statement, and can have their own monitor wait and notification methods invoked. Acquiring the monitor lock of a Condition instance, or using its monitor methods, has no specified relationship with acquiring the Lock associated with that Condition or the use of its waiting and signalling methods. It is recommended that to avoid confusion you never use Condition instances in this way, except perhaps within their own implementation.

Except where noted, passing a null value for any parameter will result in a NullPointerException being thrown.

In [ ]:
ReadWriteLock